In [54]:
from pathlib import Path
import os

import pandas as pd
import numpy as np
from data_processing import *
# hacky shit
# from data_processing.data_processing import *

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [55]:
MAIN_PATH = Path(os.getcwd()).parent

## Load data

In [56]:
org_df = pd.read_csv(os.path.join(MAIN_PATH, "data", "data.csv"))

# copied from load() function but modified to aggregate data and not drop it

org_df['czas'] = pd.to_datetime(org_df['czas'], utc=True)
org_df.set_index('czas', inplace=True)


temp_zuz = pd.read_csv(os.path.join(MAIN_PATH, "data", "temp_zuz_fixed.csv"))


temp_zuz['Czas'] = pd.to_datetime(temp_zuz['Czas'], utc=True)
temp_zuz.set_index('Czas', inplace=True)
temp_zuz = temp_zuz.rename(columns={'Czas': 'czas'})

org_df.head(3)

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf01.daca.pv,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.090454,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87
2020-09-30 22:01:00+00:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.090534,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87
2020-09-30 22:02:00+00:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.090613,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87


## Simple features based on correlation

In [57]:
df = pd.DataFrame(index=org_df.index)

In [58]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = org_df.columns[org_df.columns.str.contains('001tix')].values
df['TIX1'] = org_df[TIX1].mean(axis='columns')

In [59]:
# srednia po 'WODY df KOLEKTORÓW [°C]'
TIR = org_df.columns[org_df.columns.str.contains('tir')].values
df['TIR'] = org_df[TIR].mean(axis='columns')

In [60]:
# dodanie reg nadawy koncentratu
FCX = org_df.columns[org_df.columns.str.contains('fcx')].values
df[FCX] = org_df[FCX]

In [61]:
# sumaryczna moc cieplna
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
df[NIR] = org_df[NIR]

In [62]:
df.shape

(540492, 7)

### Aggregation and lagging

In [63]:
# Renaming columns

feature_desc_df = pd.read_csv(os.path.join(MAIN_PATH, "data_processing", "feature_desc.csv"), index_col='name')

def feature_desc(name):
        try:
            if type(name)==str:
                return feature_desc_df.loc[name]['desc']
            else:
                return feature_desc_df.loc[name]['desc'].values
        except:
            return name


col_names = list(df.columns.values)

for col_name in col_names:
    df = df.rename(columns={col_name: feature_desc(col_name)})

In [64]:
df.columns

Index(['TIX1', 'TIR', 'REG NADAWY KONCENTRATU LIW1 [Mg/h]',
       'REG NADAWY KONCENTRATU LIW2 [Mg/h]',
       'REG KONCENTRAT PRAZONY LIW3 [Mg/h]', 'REG PYL ZWROT LIW4 [Mg/h]',
       'SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]'],
      dtype='object')

In [89]:
# Aggregating
AGGREGATE_INTERVAL = 15

agg_df = aggregate(df, AGGREGATE_INTERVAL)
agg_df.head(3)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,29th_agg_REG PYL ZWROT LIW4 [Mg/h],29th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],30th_agg_TIX1,30th_agg_TIR,30th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],30th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],30th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],30th_agg_REG PYL ZWROT LIW4 [Mg/h],30th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911544,29.003188,55.366181,55.099881,11.553369,22.559934,14.981148,418.914656,28.965363,55.317309,...,22.428933,14.359413,418.990512,29.087193,55.750510,54.729549,11.996658,22.543309,14.372126,1295.0
2020-10-01 00:00:00+00:00,418.820059,29.024466,54.831780,55.003218,10.996928,20.068758,15.194276,418.823029,29.001255,54.505739,...,22.641611,15.020728,418.911544,29.003188,55.366181,55.099881,11.553369,22.559934,14.981148,1303.0
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.000000,55.224717,10.980221,19.902115,14.815123,418.780041,29.062259,55.039421,...,22.494455,15.075474,418.864860,28.950773,54.391634,54.912948,11.069167,22.592863,15.057486,1302.0


In [90]:
agg_df.shape

(8701, 218)

In [91]:
# Adding lagged featues

lagged_df = agg_df.copy()

for i in range(60 // AGGREGATE_INTERVAL):
    lagged_df = pd.concat([lagged_df, agg_df.shift(i+1).add_prefix(f'{i+1}th_agg_')], axis=1)

# delete rows with nans created by shifts
lagged_df = lagged_df.iloc[(60 // AGGREGATE_INTERVAL):, :]
    
lagged_df.head(3)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,4th_agg_29th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_29th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_30th_agg_TIX1,4th_agg_30th_agg_TIR,4th_agg_30th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],4th_agg_30th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],4th_agg_30th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],4th_agg_30th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_30th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-10-01 02:00:00+00:00,418.649147,29.096685,55.128463,54.859789,12.977070,20.988711,14.234579,418.652168,29.080250,54.657057,...,22.428933,14.359413,418.990512,29.087193,55.750510,54.729549,11.996658,22.543309,14.372126,1295.0
2020-10-01 03:00:00+00:00,418.568565,28.946375,54.579503,55.063420,12.976022,23.037937,13.733975,418.571018,28.948675,54.549533,...,22.641611,15.020728,418.911544,29.003188,55.366181,55.099881,11.553369,22.559934,14.981148,1303.0
2020-10-01 04:00:00+00:00,418.496678,28.886103,55.402461,54.960071,12.992828,22.935354,13.821118,418.498917,28.883079,54.975963,...,22.494455,15.075474,418.864860,28.950773,54.391634,54.912948,11.069167,22.592863,15.057486,1302.0


In [92]:
lagged_df.shape

(8697, 1090)

In [93]:
temp_zuz.head(5)

,temp_zuz
Czas,
2020-09-30 22:00:00+00:00,1297
2020-09-30 23:00:00+00:00,1295
2020-10-01 00:00:00+00:00,1303
2020-10-01 00:30:00+00:00,1302
2020-10-01 01:00:00+00:00,1303


In [94]:
lagged_df.head(5)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,4th_agg_29th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_29th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_30th_agg_TIX1,4th_agg_30th_agg_TIR,4th_agg_30th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],4th_agg_30th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],4th_agg_30th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],4th_agg_30th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_30th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-10-01 02:00:00+00:00,418.649147,29.096685,55.128463,54.859789,12.977070,20.988711,14.234579,418.652168,29.080250,54.657057,...,22.428933,14.359413,418.990512,29.087193,55.750510,54.729549,11.996658,22.543309,14.372126,1295.0
2020-10-01 03:00:00+00:00,418.568565,28.946375,54.579503,55.063420,12.976022,23.037937,13.733975,418.571018,28.948675,54.549533,...,22.641611,15.020728,418.911544,29.003188,55.366181,55.099881,11.553369,22.559934,14.981148,1303.0
2020-10-01 04:00:00+00:00,418.496678,28.886103,55.402461,54.960071,12.992828,22.935354,13.821118,418.498917,28.883079,54.975963,...,22.494455,15.075474,418.864860,28.950773,54.391634,54.912948,11.069167,22.592863,15.057486,1302.0
2020-10-01 05:00:00+00:00,418.446714,29.008270,54.832584,54.924843,13.025152,23.839429,14.523812,418.448100,28.996128,54.132036,...,20.012150,15.198132,418.820059,29.024466,54.831780,55.003218,10.996928,20.068758,15.194276,1303.0
2020-10-01 06:00:00+00:00,418.405872,29.166321,54.543309,54.956310,12.913086,24.023084,15.041514,418.407222,29.170246,56.319990,...,20.922734,14.612538,418.735368,29.092727,55.025156,54.940377,11.032230,21.095264,14.608455,1306.0


In [95]:
lagged_df = lagged_df.join(temp_zuz['temp_zuz'])

# dodanie temperatury żużla
lagged_df = lagged_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})

In [96]:
df = lagged_df.copy()

In [97]:
pr_df = df.copy()
df = df.dropna(axis=0)
df.shape

(8686, 1091)

In [98]:
df

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,4th_agg_29th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_30th_agg_TIX1,4th_agg_30th_agg_TIR,4th_agg_30th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],4th_agg_30th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],4th_agg_30th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],4th_agg_30th_agg_REG PYL ZWROT LIW4 [Mg/h],4th_agg_30th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],4th_agg_TEMP_ZUZ,TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-10-01 02:00:00+00:00,418.649147,29.096685,55.128463,54.859789,12.977070,20.988711,14.234579,418.652168,29.080250,54.657057,...,14.359413,418.990512,29.087193,55.750510,54.729549,11.996658,22.543309,14.372126,1295.0,1306.0
2020-10-01 03:00:00+00:00,418.568565,28.946375,54.579503,55.063420,12.976022,23.037937,13.733975,418.571018,28.948675,54.549533,...,15.020728,418.911544,29.003188,55.366181,55.099881,11.553369,22.559934,14.981148,1303.0,1303.0
2020-10-01 04:00:00+00:00,418.496678,28.886103,55.402461,54.960071,12.992828,22.935354,13.821118,418.498917,28.883079,54.975963,...,15.075474,418.864860,28.950773,54.391634,54.912948,11.069167,22.592863,15.057486,1302.0,1304.0
2020-10-01 05:00:00+00:00,418.446714,29.008270,54.832584,54.924843,13.025152,23.839429,14.523812,418.448100,28.996128,54.132036,...,15.198132,418.820059,29.024466,54.831780,55.003218,10.996928,20.068758,15.194276,1303.0,1302.0
2020-10-01 06:00:00+00:00,418.405872,29.166321,54.543309,54.956310,12.913086,24.023084,15.041514,418.407222,29.170246,56.319990,...,14.612538,418.735368,29.092727,55.025156,54.940377,11.032230,21.095264,14.608455,1306.0,1303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11 01:00:00+00:00,417.603020,27.786007,61.917540,60.725406,13.053100,29.754868,19.926717,417.612902,27.761840,60.308508,...,19.723786,419.059106,28.215657,60.585661,61.103305,13.033626,30.036720,19.768953,1301.0,1299.0
2021-10-11 02:00:00+00:00,417.532874,27.092198,0.000000,0.000000,0.000000,0.000000,4.584276,417.534295,27.174123,0.000000,...,18.624423,418.841657,27.680251,60.718664,60.777577,12.998969,29.766971,18.353176,1303.0,1237.0
2021-10-11 04:00:00+00:00,417.500589,27.080442,56.110615,57.501761,0.081717,19.863183,17.766125,417.500483,27.046829,56.335515,...,18.402394,418.559242,27.344224,60.247578,61.027001,12.943016,26.859043,18.476355,1305.0,1306.0


## Split data

In [99]:
train, val, test = split(df)

In [100]:
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

## Ridge Regression

In [101]:
ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)

Pipeline(steps=[('normalization', StandardScaler()),
                ('ridge', Ridge(alpha=1, solver='cholesky'))])

In [102]:
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

24.33033568964164

## SGD Regressor

In [103]:
sgd_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])

In [104]:
sgd_model1 = sgd_model1.fit(X_train, y_train)

ValueError: y should be a 1d array, got an array of shape (6080, 2) instead.

In [105]:
metrics.mean_squared_error(y_val, sgd_model1.predict(X_val))

AttributeError: 'NoneType' object has no attribute 'T'

In [82]:
sgd_model2 = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l1', learning_rate='adaptive'))])
sgd_model2 = sgd_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model2.predict(X_val))

2.0273438394156803e+18

In [83]:
sgd_model3 = Pipeline([('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l1', learning_rate='adaptive'))])
sgd_model3 = sgd_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model3.predict(X_val))

5.598571093454587e+26

## Spline + Ridge Regression

In [86]:
ridge_spline_model1 = Pipeline([('spline', preprocessing.SplineTransformer(n_knots=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_spline_model1 = ridge_spline_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_spline_model1.predict(X_val))

99.09126415472883

In [87]:
ridge_spline_model2 = Pipeline([('spline', preprocessing.SplineTransformer(n_knots=3, knots='quantile')),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_spline_model2 = ridge_spline_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_spline_model2.predict(X_val))

145.73859217654993

## ElasticNet Regression

In [88]:
elastic_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                           ('elasitc', linear_model.ElasticNet(max_iter=10000))])
elastic_model1 = elastic_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, elastic_model1.predict(X_val))

43.93836929530888

# Using all features, without average

### Split data

In [ ]:
all_df = org_df.dropna()
all_df

In [ ]:
train_a, val_a, test_a = split(all_df)

In [ ]:
X_train_a, X_val_a, X_test_a = train_a.drop(["temp_zuz"], axis=1), val_a.drop(["temp_zuz"], axis=1), test_a.drop(["temp_zuz"], axis=1)
y_train_a, y_val_a, y_test_a = train_a["temp_zuz"], val_a["temp_zuz"], test_a["temp_zuz"]

### Polynomial Regression

In [ ]:
poly_model_all = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                          ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model_all = poly_model_all.fit(X_train_a, y_train_a)
metrics.mean_squared_error(y_val_a, poly_model_all.predict(X_val_a))

### Poly + SGD Regression

In [ ]:
sgd_poly_model_all = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='elasticnet', learning_rate='adaptive'))])
sgd_poly_model_all = sgd_poly_model_all.fit(X_train_a, y_train_a)
metrics.mean_squared_error(y_val_a, sgd_poly_model_all.predict(X_val_a))

Wniosek: zdecydowanie nie chcemy brać wszystkich feauterów

# Adding PR to features

In [ ]:
# wrzucam o prażonkach i prob
PR = org_df.columns[org_df.columns.str.contains('pr')].values
pr_df[PR] = org_df[PR]

In [ ]:
# split data
pr_df = pr_df.dropna(axis=0)
train, val, test = split(pr_df)
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

In [ ]:
X_train

In [ ]:
y_train

### Poly + SGD Regression

In [ ]:
sgd_poly_model_pr = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model_pr = sgd_poly_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model_pr.predict(X_val))

### SGD Regression

In [ ]:
sgd_model_pr = Pipeline([('normalization', preprocessing.StandardScaler()),
                       ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_model_pr = sgd_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model_pr.predict(X_val))

### Poly + Ridge Regression

In [ ]:
ridge_poly_model_pr = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='sag'))])
ridge_poly_model_pr = ridge_poly_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr.predict(X_val))

In [ ]:
ridge_poly_model_pr1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_poly_model_pr1 = ridge_poly_model_pr1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr1.predict(X_val))

In [ ]:
ridge_poly_model_pr2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model_pr2 = ridge_poly_model_pr2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr2.predict(X_val))

### Ridge Regression

In [ ]:
ridge_model_pr1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model_pr1 = ridge_model_pr1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr1.predict(X_val))

In [ ]:
ridge_model_pr2 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='sag'))])
ridge_model_pr2 = ridge_model_pr2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr2.predict(X_val))

In [ ]:
ridge_model_pr3 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='svd'))])
ridge_model_pr3 = ridge_model_pr3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr3.predict(X_val))